<a href="https://colab.research.google.com/github/burbanomerida/Proyecto-Final---3008280040101/blob/master/Redes%20Neuronales%20-%20An%C3%A1lisis%20Divorcios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modelo de Redes Neuronales 1**

Exportar Librerías

In [26]:
!pip install pyreadstat
!pip install numpy pandas
import pyreadstat
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Input, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score







Cargar Datos

In [22]:
# Lista con los nombres de los archivos
archivos_sav = [
    "D2016_201712041525490jPevoWKMXOZPJq5KpYU9Tlj4peGmEEd.sav",
    "D2017_20181226143330lNO5qooyMLWxGu8XJLL7BXprdvTjfO0q.sav",
    "D2018_20191129151703IbgVgHc5enyxiXopUTuxcrUzC5WPkzCr.sav",
    "D2019_20201201154915Yd6FWtyVWNULqHYzwTiXcRaLAl8B8jul.sav",
    "D2020_20210730184315Yd6FWtyVWNULqHYzwTiXcRaLAl8B8jul.sav",
    "D2021_20220729171254FYBQhYtMxyPU3C9wbQbaxGTEtg107w32.sav",
    "D2022_202307282030176jUXYrCteX8JWZUbSn6B6fCXUXaZQ78F.sav",
    "D2023_20240725134712YiGUHvBDjKOx7Egk9HdU8mkc6EvieRu2.sav"
]

# Lista para almacenar los DataFrames
dataframes = []

# Leer cada archivo y almacenarlo en la lista
for archivo in archivos_sav:
    df, meta = pyreadstat.read_sav(archivo)  # Lee el archivo .sav
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

print("Número de filas y columnas:", df_combinado.shape)
print("Columnas del DataFrame:", df_combinado.columns)

# Mostrar las primeras filas del DataFrame combinado
print(df_combinado.head())


Número de filas y columnas: (59826, 21)
Columnas del DataFrame: Index(['DEPREG', 'MUPREG', 'MESREG', 'AÑOREG', 'DIAOCU', 'MESOCU', 'AÑOOCU',
       'DEPOCU', 'MUPOCU', 'EDADHOM', 'EDADMUJ', 'PPERHOM', 'PPERMUJ',
       'NACHOM', 'NACMUJ', 'ESCHOM', 'ESCMUJ', 'CIUOHOM', 'CIUOMUJ', 'PUEHOM',
       'PUEMUJ'],
      dtype='object')
   DEPREG MUPREG  MESREG  AÑOREG  DIAOCU  MESOCU  AÑOOCU  DEPOCU MUPOCU  \
0     1.0   0101    12.0  2016.0     3.0    11.0  2016.0     1.0   0101   
1     1.0   0101     8.0  2016.0    17.0     5.0  2016.0     1.0   0101   
2    17.0   1701     8.0  2016.0    11.0     8.0  2016.0     1.0   0101   
3     1.0   0101     3.0  2017.0     7.0    10.0  2016.0     1.0   0101   
4    19.0   1905     2.0  2017.0    23.0    11.0  2016.0     2.0   0207   

   EDADHOM  ...  PPERHOM  PPERMUJ  NACHOM  NACMUJ  ESCHOM  ESCMUJ  CIUOHOM  \
0    999.0  ...      9.0      9.0    32.0   320.0     9.0     9.0     98.0   
1     58.0  ...      9.0      9.0    32.0   320.0     9.0     

Procesamiento de Datos

In [37]:


# Seleccionar las columnas relevantes para la predicción
df = df_combinado[['EDADHOM', 'ESCHOM', 'ESCMUJ', 'EDADMUJ']]

# Eliminar filas con valores nulos o faltantes
df = df.dropna()

# Filtrar los valores inválidos (por ejemplo, 999)
df = df[df['EDADMUJ'] != 999]
df = df[df['EDADHOM'] != 999]

# Aplicar One-Hot Encoding a las columnas categóricas (ESCHOM y ESCMUJ)
df = pd.get_dummies(df, columns=['ESCHOM', 'ESCMUJ'], drop_first=True)

# Definir la variable dependiente (y) que es la edad de la mujer
X = df.drop('EDADMUJ', axis=1)  # Variables predictoras
y = df['EDADMUJ']  # Edad de la mujer a predecir

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos (escala de 0 a 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ahora X_train y X_test tienen las columnas de ESCMUJ y ESCHOM codificadas


Construir la Red Neuronal

In [38]:
# Crear el modelo de la red neuronal
model = Sequential()

# Capa de entrada utilizando Input
model.add(Input(shape=(X_train.shape[1],)))  # Define la forma de entrada

# Primera capa oculta con regularización
model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(Dense(64, activation=LeakyReLU(alpha=0.1)))

# Segunda capa oculta
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dense(32, activation=LeakyReLU(alpha=0.1)))

# Capa de salida
model.add(Dense(1, activation=LeakyReLU(alpha=0.1)))  # Salida para regresión

# Compilar el modelo con un learning rate ajustado
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Resumen del modelo
model.summary()

# Entrenar el modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

# Evaluar el modelo en los datos de prueba
mse = model.evaluate(X_test, y_test)
rmse = np.sqrt(mse)
print(f'Mean Squared Error en datos de prueba: {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Hacer predicciones sobre los datos de prueba
predicciones = model.predict(X_test).flatten()  # Convertir a una dimensión para comparación
print("Predicciones: ", predicciones[:10])
print("Valores reales: ", y_test.values[:10])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_27 (Dense)                     │ (None, 64)                  │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,353 (32.63 KB)

 Trainable params: 8,353 (32.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 447.9812 - val_loss: 32.8790
Epoch 2/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 31.5458 - val_loss: 30.9497
Epoch 3/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 31.6152 - val_loss: 30.8573
Epoch 4/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 30.0775 - val_loss: 29.8112
Epoch 5/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 28.7773 - val_loss: 30.7711
Epoch 6/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 29.0877 - val_loss: 29.3395
Epoch 7/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28.9022 - val_loss: 29.6363
Epoch 8/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 29.4211 - val_loss: 29.6900
Epoch 9/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28.6347 - val_loss: 30.3114
Epoch 10/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 28.7688 - val_loss: 29.6099
Epoch 11/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 28.7095 - val_loss: 30.1308
Epoch 12/200
589/5

In [39]:
# Hacer predicciones sobre los datos de prueba
predicciones = model.predict(X_test)

# Calcular las métricas de desempeño
mse = mean_squared_error(y_test, predicciones)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
r2 = r2_score(y_test, predicciones)

# Mostrar las métricas
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R² (Coeficiente de determinación): {r2}')

211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Mean Squared Error (MSE): 27.296662917869515
Root Mean Squared Error (RMSE): 5.224620839627457
Mean Absolute Error (MAE): 3.7242316167327996
R² (Coeficiente de determinación): 0.6426840911080256


**Modelo de Redes Neuronales 2**

Procesamiento de Datos

In [34]:
# Seleccionar las columnas relevantes para la predicción
df2 = df_combinado[['EDADHOM', 'ESCHOM', 'ESCMUJ', 'EDADMUJ']]

# Eliminar filas con valores nulos o faltantes
df2 = df2.dropna()

# Filtrar los valores inválidos (por ejemplo, 999)
df2 = df2[df2['EDADHOM'] != 999]
# Filtrar los valores inválidos (por ejemplo, 999)
df2 = df2[df2['EDADMUJ'] != 999]

# Aplicar One-Hot Encoding a las columnas categóricas (ESCHOM y ESCMUJ)
df2 = pd.get_dummies(df2, columns=['ESCHOM', 'ESCMUJ'], drop_first=True)

# Definir la variable dependiente (y) que es la edad de la mujer
X = df2.drop('EDADHOM', axis=1)  # Variables predictoras
y = df2['EDADHOM']  # Edad de la mujer a predecir

# Dividir el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos (escala de 0 a 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Ahora X_train y X_test tienen las columnas de ESCMUJ y ESCHOM codificadas

Construir la Red Neuronal

In [35]:
# Crear el modelo de la red neuronal
model = Sequential()

# Capa de entrada utilizando Input
model.add(Input(shape=(X_train.shape[1],)))  # Define la forma de entrada

# Primera capa oculta con regularización
model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
model.add(Dense(64, activation=LeakyReLU(alpha=0.1)))

# Segunda capa oculta
model.add(Dense(32, activation='relu', kernel_regularizer='l2'))
model.add(Dense(32, activation=LeakyReLU(alpha=0.1)))

# Capa de salida
model.add(Dense(1, activation=LeakyReLU(alpha=0.1)))  # Salida para regresión

# Compilar el modelo con un learning rate ajustado
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Resumen del modelo
model.summary()

# Entrenar el modelo
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

# Evaluar el modelo en los datos de prueba
mse = model.evaluate(X_test, y_test)
rmse = np.sqrt(mse)
print(f'Mean Squared Error en datos de prueba: {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

# Hacer predicciones sobre los datos de prueba
predicciones = model.predict(X_test).flatten()  # Convertir a una dimensión para comparación
print("Predicciones: ", predicciones[:10])
print("Valores reales: ", y_test.values[:10])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_22 (Dense)                     │ (None, 64)                  │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,353 (32.63 KB)

 Trainable params: 8,353 (32.63 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 501.3025 - val_loss: 38.2148
Epoch 2/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 38.1998 - val_loss: 36.3910
Epoch 3/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 36.5125 - val_loss: 36.5735
Epoch 4/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 35.9439 - val_loss: 35.8482
Epoch 5/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 35.5618 - val_loss: 36.2362
Epoch 6/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 36.6219 - val_loss: 36.5763
Epoch 7/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 35.9552 - val_loss: 37.1134
Epoch 8/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 35.1298 - val_loss: 35.5941
Epoch 9/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 35.5475 - val_loss: 35.7639
Epoch 10/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 35.7895 - val_loss: 35.4875
Epoch 11/200
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 35.4198 - val_loss: 35.7029
Epoch 12/200
589/5

In [36]:
# Hacer predicciones sobre los datos de prueba
predicciones = model.predict(X_test)

# Calcular las métricas de desempeño
mse = mean_squared_error(y_test, predicciones)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicciones)
r2 = r2_score(y_test, predicciones)

# Mostrar las métricas
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R² (Coeficiente de determinación): {r2}')

211/211 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Mean Squared Error (MSE): 33.7977405066738
Root Mean Squared Error (RMSE): 5.813582415918243
Mean Absolute Error (MAE): 4.029711125880812
R² (Coeficiente de determinación): 0.642141217396851
